In [1]:
import cv2
import numpy as np
from PIL import Image, ImageFilter 
import statistics 
import math
from scipy import ndimage
import matplotlib.pyplot as plt

### Blackened

In [8]:
# For image 1
def black_dots(img_path, n = 4):
    image=cv2.imread(img_path)#read in the image    
    img=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)  #RGB To Gray Scale
    _, blackAndWhite = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV) # Threshold it so image becomes binary
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, 4, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    img2 = np.zeros((labels.shape), np.uint8)
    #Filter small dotted regions. Greater the size of n more information is lost from image.
    for i in range(0, nlabels - 1):
        if sizes[i] >= n:   
            img2[labels == i + 1] = 255
    res = cv2.bitwise_not(img2) #Get the image back in black and white form from its inverted from
    Image.fromarray(res).show() #View the image
    
black_dots(r"C:\Users\1570670\Videos\Problem\DAckathon-Image_Enrichment-Dataset\Blackened\blackened-1.jpg", 4)

In [14]:
#For images 3 - 6
def remove_blackened(img_path):
    image=cv2.imread(img_path) #read in the image    
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)  #RGB To Gray Scale
    im1 = Image.fromarray(gray) #Convert image to PIL image format
    im2 = im1.filter(ImageFilter.MaxFilter(size = 3)) # apply max filter on the image to reduce blackening
    im2.show() #View the image
    
remove_blackened(r"C:\Users\1570670\Videos\Problem\DAckathon-Image_Enrichment-Dataset\Blackened\blackened-4.jpg")

### Low Resolution

In [5]:
def low_res(img_path, gamma = 2):
    img_original=cv2.imread(img_path) #read in the image     
    lookUpTable = np.empty((1,256), np.uint8) # Creating an empty lookup table for gamma function values
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255) # Fill the lookup table with gamma function values to increase contrast
    res = cv2.LUT(img_original, lookUpTable) # Lookup the pixel values in image in lookup table and replace the pixel values in image with pixel values in lookup table.
    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
    res = Image.fromarray(res) #View the image 
    res.save("test-300.jpg", dpi=(300,300))
        
low_res(r"C:\Users\Rashi\Downloads\assignment\DAckathon-Image_Enrichment-Dataset\Low Resolution\low-res-4.jpg", 2)

### Denoising

In [27]:
def denoising(img_path):
    img_original = cv2.imread(img_path) # Read original image
    dst = cv2.medianBlur(img_original, 3) # Apply median filter to remove noise from image
    Image.fromarray(dst).show() # View image
    
denoising(r"C:\Users\1570670\Videos\Problem\DAckathon-Image_Enrichment-Dataset\Noise\noise-2.jpg")

### Watermark

In [30]:
def watermark(img_path, alpha = 2.2, beta = -160):
    img_orig = cv2.imread(img_path) # Read original image    
    res = alpha * img_orig + beta #  increase brightness using alpha until the watermark vanishes and reduce brightness using beta to compensate   
    res = np.clip(res, 0, 255).astype(np.uint8) # Pixel values outside the interval are clipped to the interval edges.
    Image.fromarray(res).show() # View image
    
watermark(r"C:\Users\1570670\Videos\Problem\DAckathon-Image_Enrichment-Dataset\Watermark\watermark-1.jpg")

### Rotation

In [22]:
def rotate(img_path, angle):    
    img = cv2.imread(img_path) # Read original image 
    rotated = ndimage.rotate(img, angle) # Positive angle rotates image in anticlockwise direction and negative rotates in clockwise direction 
    Image.fromarray(rotated).show() # View image
    
rotate(r"C:\Users\1570670\Videos\Problem\DAckathon-Image_Enrichment-Dataset\Rotation\rotate-5-brokenlines.jpg", 90)

RuntimeError: invalid rotation plane specified

### Skew

In [13]:
def getAngle(a, b, c):
    # Find angle between three points
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang

def deskew(img_path, n):
    image = cv2.imread(img_path) # Read original image
    #image = cv2.imread(r'C:\Users\1570670\Videos\Problem\DAckathon-Image_Enrichment-Dataset\Skew\skew-1.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #RGB To Gray Scale
    edged = cv2.Canny(gray, 75, 200) # Extract edges from image
    contours, hier = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # Find contours of image using edges
    
    ag = []
    for c in contours:
        rect = cv2.minAreaRect(c) # Using contours find minimum area rectangle points
        box = cv2.boxPoints(rect) # Convert minimum area rectangle points to box points        
        box = np.int0(box) # convert all coordinates floating point values to int
        p1, p2, p3,p4 = box # Extracting the four corners of the box
        if (p1[0] - p3[0])>0 and (p1[0] - p2[0])> n: # Check if the first box point is the right bottom corner
            p5 = np.array([p2[0], p1[1]]) 
            angle = getAngle(p2, p1, p5) # Find the angle between line made by bottom points of rectangle with horizontal line
            ag.append(angle) # Append the angle to a list    
        elif (p3[0] - p1[1])>0 and (p4[0] - p1[0])> n: # Check if the first box point is the left bottom corner
            p5 = np.array([p4[0], p1[1]])
            angle = getAngle(p4, p1, p5) # Find the angle between line made by bottom points of rectangle with horizontal line
            ag.append(angle) # Append the angle to a list 
        
    md = statistics.median(ag) # Obtain the median of all angles to check if the image is skewed by a particular angle

    rotated = ndimage.rotate(image, -md, cval = 255) # Rotate the image by the negative of the median of angles to deskew the image
    return rotated
    #Image.fromarray(rotated).show() # View image
    
rot = deskew(r'C:\Users\1570670\Videos\Problem\DAckathon-Image_Enrichment-Dataset\Perspective\perspective-2.jpg', 20)

### Perspective

In [17]:
def order_pts(h):
    pts = np.zeros((4,2),dtype = np.float32)
    add = h.sum(axis = 1)
    pts[0] = h[np.argmin(add)]
    pts[2] = h[np.argmax(add)]
    diff = np.diff(h,axis = 1)
    pts[1] = h[np.argmin(diff)]
    pts[3] = h[np.argmax(diff)]
    return pts

def draw_polygon(event, x, y, flags, param):
    global target, img, img2, temp
    if event == cv2.EVENT_LBUTTONDOWN:        
        target[4][0] = x
        target[4][1] = y
        cv2.circle(img,(target[4][0],target[4][1]), 3, (0,0,255), 2) 
        approx = order_pts(target)        
        target[0] = approx[0]
        target[1] = approx[1]
        target[2] = approx[2]
        target[3] = approx[3]
        cv2.drawContours(img2, [np.int0(approx)], -1, (255, 255, 0), 2)
        img = img2.copy()
        img2 = temp.copy()

#def perspective(img_path):
src = cv2.imread(r'C:\Users\Rashi\Downloads\assignment\DAckathon-Image_Enrichment-Dataset\Perspective\perspective-2.jpg')
img = src.copy()
img2 = src.copy()
temp = src.copy()
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY) 
edged = cv2.Canny(gray, 75, 200)

i, contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours=sorted(contours,key=cv2.contourArea,reverse=True)[1:]

lis = []
for c in contours:
    rect = cv2.minAreaRect(c)    
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    for i in box:
        lis.append(i)        

arr = np.asarray(lis)
points = cv2.convexHull(arr)
rect = np.zeros((4, 2), dtype = "float32")
target = np.zeros((5, 2), dtype = "float32")
s = points.sum(axis = 2)
target[0] = rect[0] = points[np.argmin(s)]
target[2] = rect[2] = points[np.argmax(s)]
diff = np.diff(points, axis = 2)
target[1] = rect[1] = points[np.argmin(diff)]
target[3] = rect[3] = points[np.argmax(diff)]
    
cv2.drawContours(img, [np.int0(rect)], -1, (255, 0, 0), 2)
clone = img.copy()
height, width = img.shape[:2]
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', width, height)
cv2.setMouseCallback("image", draw_polygon) 
    
while True: 
    # display the image and wait for a keypress 
    cv2.imshow("image", img)    
    key = cv2.waitKey(1) & 0xFF
  
    # press 'r' to reset the window 
    if key == ord("r"): 
        img = clone.copy()
        img2 = src.copy()        
        target[0] = rect[0]
        target[1] = rect[1]
        target[2] = rect[2]
        target[3] = rect[3]      
      
    # if the 'q' key is pressed, break from the loop 
    if key == ord("q"): 
        break
        
cv2.destroyAllWindows()        
rect[0] = target[0]
rect[1] = target[1]
rect[2] = target[2]
rect[3] = target[3]

box = np.float32(np.int0(rect))   
pts = np.float32([[0,0],[src.shape[1],0],[src.shape[1],src.shape[0]],[0,src.shape[0]]])  #map to 800*800 target window
op = cv2.getPerspectiveTransform(box,pts)  #get the top or bird eye view effect
dst = cv2.warpPerspective(src,op,(src.shape[1],src.shape[0]))
dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
Image.fromarray(dst).show()

In [21]:
def line(x1, x2, x_fin):
    y_diff = x2[1] - x1[1]
    x_diff = x2[0] - x1[0]
    try:
        m = y_diff/x_diff
    except:        
        return x_fin[0]
    x = ((-x_fin[1])/m) + x_fin[0]
    return x

def intersection(M1, M2, X1, X2):
    if (M1[0][0] - M1[1][0]) == 0 and (M2[0][1] - M2[1][1]) == 0:
        x = X1[0]
        y = 0
        return x, y
    elif (M1[0][0] - M1[1][0]) == 0:
        x = X1[0]
        m2 = (M2[0][1] - M2[1][1])/(M2[0][0] - M2[1][0])
        c = -(m2* X2[0])
        y = (m2*x) + c
        return x, y
    elif (M2[0][1] - M2[1][1]) == 0:
        y = 0        
        m1 = (M1[0][1] - M1[1][1])/(M1[0][0] - M1[1][0])
        c = -(m1* X1[0]) + X1[1]
        x = (-c)/m1
        return x, y
    else:
        m1 = (M1[0][1] - M1[1][1])/(M1[0][0] - M1[1][0])
        m2 = (M2[0][1] - M2[1][1])/(M2[0][0] - M2[1][0]) 
        m_diff = m2-m1
        det = X1[1] + (m2* X2[0]) - (m1*X1[0])
        x = det/m_diff
        y = m2*(x-X2[0])
        return x, y

def perspective(img_path, angle):
    src = cv2.imread(img_path) # Read original image
    #src = cv2.imread(r'C:\Users\1570670\Videos\Problem\DAckathon-Image_Enrichment-Dataset\Perspective\perspective-2.jpg')
    #Image.fromarray(src).show()
    orig_rotated = ndimage.rotate(src, angle) # Rotate the image so that the contents form a parallel trapezoid with smaller side on top
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY) # convert to grayscale
    edged = cv2.Canny(gray, 75, 200)
    rotated = ndimage.rotate(edged, angle)

    i, contours, hierarchy = cv2.findContours(rotated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours=sorted(contours,key=cv2.contourArea,reverse=True)[1:]

    lis = []
    zero = np.zeros((edged.shape[0], edged.shape[1], 3), np.uint8)
    for c in contours:
        rect = cv2.minAreaRect(c)    
        box = cv2.boxPoints(rect)
        box = np.int0(box)         
        for i in box:            
            lis.append(i)     

    arr = np.asarray(lis)
    points = cv2.convexHull(arr)

    rect = np.zeros((4, 2), dtype = "float32")
    pts_fin = np.zeros((4, 2), dtype = "float32")
    s = points.sum(axis = 2)

    rect[0] = points[np.argmin(s)]
    rect[2] = points[np.argmax(s)]

    diff = np.diff(points, axis = 2)
    rect[1] = points[np.argmin(diff)]
    rect[3] = points[np.argmax(diff)]
    
    pts_fin[2][0] = rotated.shape[1] - 1
    pts_fin[2][1] = rotated.shape[0] -1
    pts_fin[3][0] = 0
    pts_fin[3][1] = rotated.shape[0] -1

    if rect[0][1] < rect[1][1]:
        pts_fin[0][0] = line(rect[0], rect[3], pts_fin[3])
        pts_fin[0][1] = 0
        pts_fin[1][0], pts_fin[1][1] = intersection([rect[1], rect[2]], [rect[0], rect[1]], pts_fin[2], pts_fin[0])
    
    else:
        pts_fin[1][0] = line(rect[1], rect[2], pts_fin[2])
        pts_fin[1][1] = 0
        pts_fin[0][0], pts_fin[0][1] = intersection([rect[0], rect[3]], [rect[0], rect[1]], pts_fin[3], pts_fin[1])
    
    box = np.float32(np.int0(pts_fin))
    pts=np.float32([[0,0],[rotated.shape[1],0],[rotated.shape[1],rotated.shape[0]],[0,rotated.shape[0]]])  #map to 800*800 target window
    orig_rotated = ndimage.rotate(src, angle) 
    op=cv2.getPerspectiveTransform(box,pts)  #get the top or bird eye view effect
    dst=cv2.warpPerspective(orig_rotated,op,(rotated.shape[1],rotated.shape[0]))
    dst = ndimage.rotate(dst, -(angle))

    Image.fromarray(dst).show()
    
perspective(r'C:\Users\1570670\Videos\Problem\DAckathon-Image_Enrichment-Dataset\Perspective\perspective-2.jpg',90)

RuntimeError: invalid rotation plane specified